# Getting Started with Kubernetes on Atomic Host

### Objectives

Create an Ansible Playbook that:

1.   Compiles a cryptocurrency from source
2.   Deploya on Atomic Host OS in AWS EC2

1.   Uses Kubernetes for orchestratioin
2.   Uses Ansible-Container
*   Cryptographic verification
*   Add a unique feature/integration








### Components



*   Flannel for network overlay
*   Docker for containerization

*   Kubernetes for container orchestration
*   Fedora for immutable operating system

*   Ethereum Go for cryptocurrency

*   Flannel for network overlay
*   Ansible-Playbook for automation


*   Jupyter Notebook for documentation and additional automation
*   Amazon Web Services for cloud infrastructure


*   Github for version control












# Table of Contents

>[Getting Started with Kubernetes on Atomic Host](#scrollTo=1RJmwc_mPfBm)

>>>[Objectives](#scrollTo=VK__AuHafG8I)

>>>[Components](#scrollTo=Ceix_OlGfviU)

>[Step One: AWS Prerequisites](#scrollTo=ahpfT_x4gvpw)

>[Step Two: Launch Fedora EC2 Instance](#scrollTo=WbIhllF3he8W)

>[Step Three: Create a Local Docker-Hub Registry](#scrollTo=Q5OJpN2Uhd0Q)

>>>[Reload the systemd daemon and start the new local-registry service.](#scrollTo=qwJ0WrAskdvE)

>[Step 4: Configure Kubernetes Master](#scrollTo=VkA6rpTTlKlU)

>>[Configure etcd](#scrollTo=7aSVyy8RlXn8)

>>[Generate certificates](#scrollTo=-SazVklvll4i)

>>>[Generate a CA. (--batch set automatic mode. --req-cn default CN to use.)](#scrollTo=y1dSWgXhl7gq)

>>>[Generate server certificate and key](#scrollTo=l5jyl76imGX6)

>>>[Copy pki/ca.crt, pki/issued/kubernetes-master.crt, and pki/private/kubernetes-master.key to /etc/kubernetes/certs.](#scrollTo=in8IPz_kmNU8)

>[Configure services](#scrollTo=h7vp6TDdmbBa)

>>[Apiserver service configuration](#scrollTo=BI5J5He8nGVc)

>>[Controller-manager service configuration](#scrollTo=PfkQUhkSnxZo)

>>>[Enable and start the Kubernetes services.](#scrollTo=bSR5rby1oCkQ)

>[Configuring the Flannel overlay network](#scrollTo=2EO2SpJ7oJW2)

>>[Create a keyname specific to this cluster](#scrollTo=cFlVuBdmof40)

>>[Make sure we have the right config](#scrollTo=rX--bYiioo2w)

>[Atomic Nodes](#scrollTo=n1ArasZ3oz36)

>>>[As a good practice, update to the latest available Atomic tree.](#scrollTo=9yPV446npCG4)

>>[Configuring Docker to use the cluster registry cache](#scrollTo=pdiuBauxpLDM)

>>[Configuring Docker to use the Flannel overlay](#scrollTo=Ct_VaLm_pYA-)

>>[Configuring Kubernetes nodes](#scrollTo=LJLFVtrFpmog)

>>[Set the location of the etcd server](#scrollTo=__RcFr4Pp45o)

>>[Set the cluster CIDR for Kubernetes Proxy.](#scrollTo=PJwiuOp_qMlU)

>>[Reload](#scrollTo=rSRtamWRqY46)

>[Confirm network configuration and cluster health](#scrollTo=30dsa3WeqlKI)

>[Repeat these steps on the other 3 nodes to complete the cluster configuration.](#scrollTo=YeaFb2J6q68u)

>[Exploring Kubernetes](#scrollTo=ukacb7ATrBYi)

>>>[To get the pod up and running, use kubectl create](#scrollTo=nhbR2MQGsHri)

>>>[To check the status of the containers using kubectl get](#scrollTo=FLn3T4MBsL7Y)

>>>[Check to see which node it's running on](#scrollTo=vIMzxP_dsTLY)



# Step One: AWS Prerequisites

In [0]:
!ansible-playbook aws configure

In [0]:
!ansible-playbook aws-keys.yml

In [0]:
!ansible-playbook aws-iam.yml

In [0]:
!ansible-playbook aws-sec.yml

In [0]:
!ansible-playbook vpc.yml

# Step Two: Launch Fedora EC2 Instance

This will be our master node. Etcd, services, and the docker cashe will be located here.

In [0]:
!ansible-playbook fedora.yml

In [0]:
sudo atomic host upgrade --reboot

# Step Three: Create a Local Docker-Hub Registry

Create a named container from the Docker Hub registry image, exposing the standard Docker Hub port from the container via the host.  We're using a local host directory as a persistence layer for the images that get cached for use.  The other environment variables passed in to the registry set the source registry.

In [0]:
sudo docker create -p 5000:5000 \ -v /var/lib/local-registry:/var/lib/registry \ -e 
REGISTRY_STORAGE_FILESYSTEM_ROOTDIRECTORY=/var/lib/registry \ -e 
REGISTRY_PROXY_REMOTEURL=https://registry-1.docker.io \ --name=local-registry registry:2

Note: We need to change the SELinux context on the directory that docker created for our persistence volume.

In [0]:
sudo mkdir -p /var/lib/local-registry

In [0]:
sudo chcon -Rvt svirt_sandbox_file_t /var/lib/local-registry

### Reload the systemd daemon and start the new local-registry service.

In [0]:
sudo vi /etc/systemd/system/local-registry.service


In [0]:
[Unit]
Description=Local Docker Mirror registry cache
Requires=docker.service
After=docker.service

In [0]:
[Service]
Restart=on-failure
RestartSec=10
ExecStart=/usr/bin/docker start -a %p
ExecStop=-/usr/bin/docker stop -t 2 %p

In [0]:
[Install]
WantedBy=multi-user.target

In [0]:
sudo systemctl daemon-reload

In [0]:
sudo systemctl enable local-registry

In [0]:
sudo systemctl start local-registry

# Step 4: Configure Kubernetes Master

## Configure etcd

In [0]:
sudo vi /etc/etcd/etcd.conf
ETCD_LISTEN_CLIENT_URLS="http://0.0.0.0:2379,http://0.0.0.0:4001"
ETCD_ADVERTISE_CLIENT_URLS="http://192.168.122.10:2379,http://192.168.122.10:4001"

## Generate certificates

In [0]:
curl -L -O https://storage.googleapis.com/kubernetes-release/easy-rsa/easy-rsa.tar.gz

In [0]:
tar xzf easy-rsa.tar.gz

In [0]:
cd easy-rsa-master/easyrsa3

In [0]:
./easyrsa init-pk

### Generate a CA. (--batch set automatic mode. --req-cn default CN to use.)

In [0]:
MASTER_IP=192.168.122.10

In [0]:
./easyrsa --batch "--req-cn=${MASTER_IP}@`date +%s`" build-ca nopass

### Generate server certificate and key

In [0]:
./easyrsa --subject-alt-name="IP:${MASTER_IP}" build-server-full server nopass

### Copy pki/ca.crt, pki/issued/kubernetes-master.crt, and pki/private/kubernetes-master.key to /etc/kubernetes/certs.

In [0]:
sudo mkdir /etc/kubernetes/certs

In [0]:
for i in {pki/ca.crt,pki/issued/server.crt,pki/private/server.key}; do sudo cp $i /etc/kubernetes/certs; done

In [0]:
sudo chown -R kube:kube /etc/kubernetes/certs

# Configure services

```
Services
config
apiserver
controller-manager
scheduler
```



We'll be setting up the etcd store that Kubernetes will use.  We're using a single local etcd service, so we'll point that at the master on the standard port.  We'll also set up how the services find the apiserver.


```
# Comma separated list of nodes in the etcd cluster
KUBE_ETCD_SERVERS="--etcd_servers=http://192.168.122.10:2379"

# How the controller-manager, scheduler, and proxy find the kube-apiserver
KUBE_MASTER="--master=http://192.168.122.10:6443"```



In [0]:
sudo vi /etc/kubernetes/config

## Apiserver service configuration

The apiserver needs to be set to listen on all IP addresses, instead of just localhost.

```
# The address on the local server to listen to.
KUBE_API_ADDRESS="--insecure-bind-address=0.0.0.0"

```



In [0]:
sudo vi /etc/kubernetes/apiserver

If you need to modify the set of IPs that Kubernetes assigns to services, change the KUBE_SERVICE_ADDRESSES value. Since this guide is using the 192.168.122.0/24 and 172.16.0.0/12 networks, we can leave the default.  This address space needs to be unused elsewhere, but doesn't need to be reachable from either of the other networks.

```
# Address range to use for services
KUBE_SERVICE_ADDRESSES="--service-cluster-ip-range=10.254.0.0/16"
```



We'll also add parameters for the certificates we generated earlier.

```
# Add your own!
KUBE_API_ARGS="--tls-cert-file=/etc/kubernetes/certs/server.crt --tls-private-key-file=/etc/kubernetes/certs/server.key --client-ca-file=/etc/kubernetes/certs/ca.crt --service-account-key-file=/etc/kubernetes/certs/server.crt"```



## Controller-manager service configuration

The controller-manager also needs parameters for the certificates we generated.

```
# Add your own!
KUBE_CONTROLLER_MANAGER_ARGS="--service-account-private-key-file=/etc/kubernetes/certs/server.key --root-ca-file=/etc/kubernetes/certs/ca.crt"
```



In [0]:
sudo vi /etc/kubernetes/controller-manager

### Enable and start the Kubernetes services.

In [0]:
sudo systemctl enable etcd flanneld kube-apiserver kube-controller-manager kube-scheduler

In [0]:
sudo systemctl start etcd flanneld kube-apiserver kube-controller-manager kube-scheduler

# Configuring the Flannel overlay network



```
{
"Network": "172.16.0.0/12",
"SubnetLen": 24,
"Backend": {
"Type": "vxlan"
}
}
```



In [0]:
vi flanneld-conf.json

## Create a keyname specific to this cluster

In [0]:
curl -L http://localhost:2379/v2/keys/atomic.io/network/config -XPUT --data-urlencode value@flanneld-conf.json

## Make sure we have the right config

```
{
"action": "get",
"node": {
"createdIndex": 11,
"key": "/atomic.io/network/config",
"modifiedIndex": 11,
"value": "{\n  \"Network\": \"172.16.0.0/12\",\n  \"SubnetLen\": 24,\n  \"Backend\": {\n    \"Type\": \"vxlan\"\n  }\n}\n\n"
}
}
```



In [0]:
curl -L http://localhost:2379/v2/keys/atomic.io/network/config | python -m json.tool

# Atomic Nodes

We'll be configuring Docker to use Flannel and our cache for configuring the Kubernetes services.  These nodes will act as the workers and run Pods and containers.  You can repeat this on as many nodes as you like to provide resources to the cluster.  In this guide, we'll set up 4 nodes.

### As a good practice, update to the latest available Atomic tree.

In [0]:
sudo atomic host upgrade

In [0]:
sudo systemctl reboo

## Configuring Docker to use the cluster registry cache

Add the local cache registry running on the master to the docker options that get pulled into the systemd unit file.

```
OPTIONS='--registry-mirror=http://192.168.122.10:5000 --selinux-enabled --log-driver=journald'
```



In [0]:
sudo vi /etc/sysconfig/docker

## Configuring Docker to use the Flannel overlay

To set up flanneld, we need to tell the local flannel service where to find the etcd service serving up the config. We also give it the right key to find the networking values for this cluster.


```
# etcd url location.  Point this to the server where etcd runs
FLANNEL_ETCD_ENDPOINTS="http://192.168.122.10:2379"

# etcd config key.  This is the configuration key that flannel queries
# For address range assignment
FLANNEL_ETCD_PREFIX="/atomic.io/network"
```



In [0]:
sudo vi /etc/sysconfig/flanneld

## Configuring Kubernetes nodes

The address entry in the kubelet config file must match the KUBLET_ADDRESSES entry on the master.   If hostnames are used, this also must match output of hostname -f on the node.  We're using the eth0 IP address like we did on the master.

```
# The address for the info server to serve on (set to 0.0.0.0 or "" for all interfaces)
KUBELET_ADDRESS="--address=192.168.122.11"

# You may leave this blank to use the actual hostname
KUBELET_HOSTNAME="--hostname-override=192.168.122.11"

# location of the api-server
KUBELET_API_SERVER="--api-servers=http://192.168.122.10:8080"

```



In [0]:
sudo vi /etc/kubernetes/kubelet

## Set the location of the etcd server



```
# How the controller-manager, scheduler, and proxy find the kube-apiserver
KUBE_MASTER="--master=http://192.168.122.10:8080"
```



In [0]:
sudo vi /etc/kubernetes/config

## Set the cluster CIDR for Kubernetes Proxy.



```
# Add your own!
KUBE_PROXY_ARGS="--cluster-cidr=10.254.0.0/16"
```



In [0]:
sudo vi /etc/kubernetes/proxy

## Reload

In [0]:
sudo iptables --policy FORWARD ACCEPT

In [0]:
sudo systemctl daemon-reload

In [0]:
sudo systemctl enable flanneld kubelet kube-proxy

In [0]:
sudo systemctl reboot

# Confirm network configuration and cluster health

Once all of your services are started, the networking should look something like what's below.  You'll see the Flannel device that shows the selected range for this host and the docker0 bridge that has a specific subnet assigned.



```
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9001 qdisc pfifo_fast state UP group default qlen 1000
link/ether 0a:45:46:8d:6a:de brd ff:ff:ff:ff:ff:ff
inet 10.4.0.120/24 brd 10.4.0.255 scope global dynamic eth0
valid_lft 3570sec preferred_lft 3570sec
inet6 fe80::845:46ff:fe8d:6ade/64 scope link
valid_lft forever preferred_lft forever
3: flannel.1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 8951 qdisc noqueue state UNKNOWN group default
link/ether 1a:50:6d:23:5d:a2 brd ff:ff:ff:ff:ff:ff
inet 172.16.36.0/12 scope global flannel.1
valid_lft forever preferred_lft forever
inet6 fe80::1850:6dff:fe23:5da2/64 scope link
valid_lft forever preferred_lft forever
5: docker0: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc noqueue state DOWN group default
link/ether 56:84:7a:fe:97:99 brd ff:ff:ff:ff:ff:ff
inet 172.16.36.1/24 scope global docker0
valid_lft forever preferred_lft forever

```



In [0]:
sudo systemctl status flanneld docker kubelet kube-proxy

In [0]:
ip a

# Repeat these steps on the other 3 nodes to complete the cluster configuration.

# Exploring Kubernetes

In [0]:
kubectl get node

We can now create a simple Kubernetes pod to schedule a workload.

We'll create a simple nginx pod definition on the master.  You can use JSON or YAML to create pods, we'll use YAML.

```
apiVersion: v1
kind: Pod
metadata:
name: www
spec:
containers:
  name: nginix
    image: nginx
    ports:
      containerPort: 80
      hostport: 8080
```



In [0]:
vi kube-nginx.yml

### To get the pod up and running, use kubectl create

In [0]:
kubectl create -f kube-nginx.yml

### To check the status of the containers using kubectl get

In [0]:
kubectl get pod

### Check to see which node it's running on

In [0]:
kubectl describe pods www | grep Node

Point a web browser at the host Kubernetes created the container on. Use port 8080, since that was the host port we connected to the container port 80 in the pod definition. You should see the nginx welcome page.

You've now created and scheduled your first kubernetes pod.  You can explore the kubernetes documentation for more information on how to build pods and services, and checkout these ansible scripts for a fuller-featured cluster that includes kubernetes addons such as dns.